# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""66526714496212…","""29692933595473…","""../../../../da…","""14235259546565…","""11209164150765…",false
1.4856e19,"""97482130749936…","""11964963790368…","""../../../../da…","""14235259546565…","""12369327655890…",true
5.6458e17,"""15972111734345…","""47691578853928…","""../../../../da…","""64561219271934…","""11209164150765…",false
4.0723e18,"""97482130749936…","""88670864045259…","""../../../../da…","""18122970989605…","""11209164150765…",false
1.4677e19,"""66526714496212…","""14455781705313…","""../../../../da…","""18122970989605…","""17646959532832…",true
1.6330e19,"""15972111734345…","""14013775461747…","""../../../../da…","""13675369541481…",null,false
8.7675e18,"""15972111734345…","""43359284942717…","""../../../../da…","""18122970989605…","""17646959532832…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""15617293385758…","""17371285973342…","""../../../../da…","""FirstMortgage3…","""10620329084147…","""Rejected"""
1.4856e19,"""17046245443677…","""14316508188531…","""../../../../da…","""FirstMortgage3…","""89699665134391…","""Accepted"""
5.6458e17,"""14132566875036…","""12313279250907…","""../../../../da…","""MoneyMarketSav…","""10620329084147…","""Rejected"""
4.0723e18,"""17046245443677…","""97686161037651…","""../../../../da…","""BasicChecking""","""10620329084147…","""Rejected"""
1.4677e19,"""15617293385758…","""44440556749278…","""../../../../da…","""BasicChecking""","""64013245854424…","""Accepted"""
1.6330e19,"""14132566875036…","""18248418998918…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""14132566875036…","""11030385617665…","""../../../../da…","""BasicChecking""","""64013245854424…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""87369257940103…","""10535316624230…","""../../../../da…","""FirstMortgage3…","""13227709960437…","""Rejected"""
1.4856e19,"""40416675154065…","""17144553996205…","""../../../../da…","""FirstMortgage3…","""13284121494191…","""Accepted"""
5.6458e17,"""18431311201394…","""71258393424999…","""../../../../da…","""MoneyMarketSav…","""13227709960437…","""Rejected"""
4.0723e18,"""40416675154065…","""85065603375564…","""../../../../da…","""BasicChecking""","""13227709960437…","""Rejected"""
1.4677e19,"""87369257940103…","""68441453490210…","""../../../../da…","""BasicChecking""","""14494256486233…","""Accepted"""
1.6330e19,"""18431311201394…","""13630269744742…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""18431311201394…","""12604808447557…","""../../../../da…","""BasicChecking""","""14494256486233…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""83011734515338…","""17650633749912…","""FirstMortgage3…","""69360374940376…","""Rejected"""
1.4856e19,"""89964691861830…","""85899333162546…","""FirstMortgage3…","""34295806061795…","""Accepted"""
5.6458e17,"""58864247645148…","""11442970269565…","""MoneyMarketSav…","""69360374940376…","""Rejected"""
4.0723e18,"""89964691861830…","""27953477958887…","""BasicChecking""","""69360374940376…","""Rejected"""
1.4677e19,"""83011734515338…","""10121404329141…","""BasicChecking""","""12430035930158…","""Accepted"""
1.6330e19,"""58864247645148…","""17404628342792…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""58864247645148…","""12905364428377…","""BasicChecking""","""12430035930158…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'